# ENSEMBLE OF RES-NET-34 & EFFICIENT NET MODELS

In [1]:
import os
import cv2
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Input, Normalization, Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.metrics import BinaryAccuracy, FalsePositives, FalseNegatives, TrueNegatives, TruePositives, Precision, Recall, F1Score, AUC
from tensorflow.keras.regularizers import L2
import sklearn

In [2]:
CONFIGURATION = {
    "CLASS_NAMES" : ['angry', 'happy', 'sad'],
    "BATCH_SIZE" : 32,
    "IMAGE_SIZE" : 256,
    "LEARNING_RATE" : 0.01,
    "N_EPOCHS" : 60,
    "DROPOUT_RATE": 0.0,
    "REGULARIZATION_RATE" : 0.0,
    "N_FILTERS" : 6,
    "KERNEL_SIZE" : 3,
    "N_STRIDES" : 1,
    "POOL_SIZE" : 2,
    "N_DENSE_1" : 128,
    "N_DENSE_2" : 32,
    "NUM_CLASSES" : 3
}

trainDirectory = "/Users/aman/Documents/Work/Machine Learning/Computer-Vision-TensorFlow/Human-Emotions-Detection/Dataset/Emotions Dataset/Emotions Dataset/train"
testDirectory = "/Users/aman/Documents/Work/Machine Learning/Computer-Vision-TensorFlow/Human-Emotions-Detection/Dataset/Emotions Dataset/Emotions Dataset/test"

trainDataset = tf.keras.utils.image_dataset_from_directory(
    trainDirectory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=0.2,
    subset='training',
)

valDataset = tf.keras.utils.image_dataset_from_directory(
    trainDirectory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=0.2,
    subset='validation',
)

testDataset = tf.keras.utils.image_dataset_from_directory(
    testDirectory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=None,
    subset=None,
)

trainDataset = trainDataset.prefetch(tf.data.AUTOTUNE)
testDataset = testDataset.prefetch(tf.data.AUTOTUNE)
valDataset = valDataset.prefetch(tf.data.AUTOTUNE)

Found 6799 files belonging to 3 classes.
Using 5440 files for training.
Found 6799 files belonging to 3 classes.
Using 1359 files for validation.


2025-01-31 11:38:59.398684: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-01-31 11:38:59.398719: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-01-31 11:38:59.398728: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-01-31 11:38:59.398745: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-31 11:38:59.398758: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 2278 files belonging to 3 classes.


## Training Res-Net-34 Model

In [3]:
class CustomConv2D(tf.keras.layers.Layer):
  def __init__(self, n_filters, kernel_size, n_strides, padding = 'valid'):
    super(CustomConv2D, self).__init__(name = 'custom_conv2d')

    self.conv = Conv2D(
        filters = n_filters,
        kernel_size = kernel_size,
        activation = 'relu',
        strides = n_strides,
        padding = padding)

    self.batch_norm = tf.keras.layers.BatchNormalization()

  def call(self, x, training = True):

    x = self.conv(x)
    x = self.batch_norm(x, training=training)

    return x
  
class ResidualBlock(tf.keras.layers.Layer):
  def __init__(self, n_channels, n_strides = 1):
    super(ResidualBlock, self).__init__(name = 'res_block')

    self.sizeChanged = (n_strides != 1)

    self.custom_conv_1 = CustomConv2D(n_channels, 3, n_strides, padding = "same")
    self.custom_conv_2 = CustomConv2D(n_channels, 3, 1, padding = "same")

    self.activation = tf.keras.layers.Activation('relu')

    if self.sizeChanged:
      self.custom_conv_3 = CustomConv2D(n_channels, 1, n_strides)

  def call(self, input, training):

    x = self.custom_conv_1(input, training=training)
    x = self.custom_conv_2(x, training=training)

    if self.sizeChanged:
      input = self.custom_conv_3(input, training=training)
      x_added = tf.keras.layers.Add()([x, input])
    else:
      x_added = tf.keras.layers.Add()([x, input])

    return self.activation(x_added)


In [5]:
class ResNet34(tf.keras.models.Model):
  def __init__(self,):
    super(ResNet34, self).__init__(name = 'resnet_34')

    self.conv_1 = CustomConv2D(64, 7, 2, padding = 'same')
    self.max_pool = MaxPooling2D(3,2)

    self.conv_2_1 = ResidualBlock(64)
    self.conv_2_2 = ResidualBlock(64)
    self.conv_2_3 = ResidualBlock(64)

    self.conv_3_1 = ResidualBlock(128, 2)
    self.conv_3_2 = ResidualBlock(128)
    self.conv_3_3 = ResidualBlock(128)
    self.conv_3_4 = ResidualBlock(128)

    self.conv_4_1 = ResidualBlock(256, 2)
    self.conv_4_2 = ResidualBlock(256)
    self.conv_4_3 = ResidualBlock(256)
    self.conv_4_4 = ResidualBlock(256)
    self.conv_4_5 = ResidualBlock(256)
    self.conv_4_6 = ResidualBlock(256)

    self.conv_5_1 = ResidualBlock(512, 2)
    self.conv_5_2 = ResidualBlock(512)
    self.conv_5_3 = ResidualBlock(512)

    self.global_pool = tf.keras.layers.GlobalAveragePooling2D()

    self.fc_3 = tf.keras.layers.Dense(CONFIGURATION["NUM_CLASSES"], activation = 'softmax')

  def call(self, x, training = True):
    x = self.conv_1(x)
    x = self.max_pool(x)

    x = self.conv_2_1(x, training=training)
    x = self.conv_2_2(x, training=training)
    x = self.conv_2_3(x, training=training)

    x = self.conv_3_1(x, training=training)
    x = self.conv_3_2(x, training=training)
    x = self.conv_3_3(x, training=training)
    x = self.conv_3_4(x, training=training)

    x = self.conv_4_1(x, training=training)
    x = self.conv_4_2(x, training=training)
    x = self.conv_4_3(x, training=training)
    x = self.conv_4_4(x, training=training)
    x = self.conv_4_5(x, training=training)
    x = self.conv_4_6(x, training=training)

    x = self.conv_5_1(x, training=training)
    x = self.conv_5_2(x, training=training)
    x = self.conv_5_3(x, training=training)

    x = self.global_pool(x)

    return self.fc_3(x)

resNet34Model = ResNet34()
resNet34Model(tf.zeros([1, 256, 256, 3]), training=True)
resNet34Model.summary()

Model: "resnet_34"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ custom_conv2d (CustomConv2D)    │ ?                      │         9,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block (ResidualBlock)       │ ?                      │        74,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block (ResidualBlock)       │ ?                      │        74,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block (ResidualBlock)       │ ?                      │        74,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block (ResidualBlock)       │ ?                      │       231,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block (ResidualBlock)       │ ?                      │       296,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block (ResidualBlock)       │ ?                      │       296,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block (ResidualBlock)       │ ?                      │       296,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block (ResidualBlock)       │ ?                      │       921,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block (ResidualBlock)       │ ?                      │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block (ResidualBlock)       │ ?                      │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block (ResidualBlock)       │ ?                      │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block (ResidualBlock)       │ ?                      │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block (ResidualBlock)       │ ?                      │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block (ResidualBlock)       │ ?                      │     3,677,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block (ResidualBlock)       │ ?                      │     4,723,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block (ResidualBlock)       │ ?                      │     4,723,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (1, 3)                 │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,311,747 (81.30 MB)

 Trainable params: 21,294,723 (81.23 MB)

 Non-trainable params: 17,024 (66.50 KB)

In [6]:
lossFunction = tf.keras.losses.CategoricalCrossentropy()
METRICS = [tf.keras.metrics.CategoricalAccuracy(name="accuracy"), tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="top_k_accuracy")]

In [7]:
resNet34Model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=CONFIGURATION["LEARNING_RATE"]),
    loss = lossFunction,
    metrics=METRICS
)

In [8]:
history = resNet34Model.fit(
    trainDataset,
    validation_data = valDataset,
    epochs = CONFIGURATION['N_EPOCHS'],
    verbose=1,
)

Epoch 1/60


2025-01-31 11:40:12.114561: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


170/170 ━━━━━━━━━━━━━━━━━━━━ 109s 520ms/step - accuracy: 0.3989 - loss: 2.3740 - top_k_accuracy: 0.7198 - val_accuracy: 0.3348 - val_loss: 154.0747 - val_top_k_accuracy: 0.7123
Epoch 2/60
170/170 ━━━━━━━━━━━━━━━━━━━━ 85s 498ms/step - accuracy: 0.4519 - loss: 1.2262 - top_k_accuracy: 0.7496 - val_accuracy: 0.5121 - val_loss: 1.1423 - val_top_k_accuracy: 0.7579
Epoch 3/60
170/170 ━━━━━━━━━━━━━━━━━━━━ 85s 498ms/step - accuracy: 0.4943 - loss: 1.0769 - top_k_accuracy: 0.7788 - val_accuracy: 0.5004 - val_loss: 1.3701 - val_top_k_accuracy: 0.7866
Epoch 4/60
170/170 ━━━━━━━━━━━━━━━━━━━━ 86s 506ms/step - accuracy: 0.5107 - loss: 1.0347 - top_k_accuracy: 0.8019 - val_accuracy: 0.5018 - val_loss: 1.1056 - val_top_k_accuracy: 0.7815
Epoch 5/60
170/170 ━━━━━━━━━━━━━━━━━━━━ 90s 530ms/step - accuracy: 0.5326 - loss: 0.9963 - top_k_accuracy: 0.8148 - val_accuracy: 0.5342 - val_loss: 1.0242 - val_top_k_accuracy: 0.8102
Epoch 6/60
170/170 ━━━━━━━━━━━━━━━━━━━━ 87s 508ms/step - accuracy: 0.5582 - loss: 0

## Leveraging Efficient Net Pre-Trained Model

In [9]:
# Loading Pre-Trained Efficient Net B4 Model trained on ImageNet Dataset
backbone = tf.keras.applications.EfficientNetB4(
    include_top = False, # Include Classifier or Not
    weights = 'imagenet',
    input_shape = (CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"], 3)
)
# Freezing The Model Weights so That They Do Not Update
backbone.trainable = False

In [10]:
# Resize-Rescale Layer will be added to Model itself for easy deployment
efficientNetModel = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None, None, 3)),
    backbone, # Feature Extractor Part of Model
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(CONFIGURATION['N_DENSE_1'], activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(CONFIGURATION['N_DENSE_2'], activation='relu'),
    tf.keras.layers.Dense(CONFIGURATION['NUM_CLASSES'], activation='softmax')
])
efficientNetModel.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb4 (Functional)     │ (None, None, None,     │    17,673,823 │
│                                 │ 1792)                  │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1792)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │       229,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_36          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,908,066 (68.31 MB)

 Trainable params: 233,987 (914.01 KB)

 Non-trainable params: 17,674,079 (67.42 MB)

In [11]:
efficientNetModel.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=CONFIGURATION["LEARNING_RATE"]),
    loss = lossFunction,
    metrics=METRICS
)

In [12]:
history2 = efficientNetModel.fit(
    trainDataset,
    validation_data = valDataset,
    epochs = CONFIGURATION['N_EPOCHS'],
    verbose=1,
)

Epoch 1/60
170/170 ━━━━━━━━━━━━━━━━━━━━ 94s 484ms/step - accuracy: 0.6761 - loss: 1.1505 - top_k_accuracy: 0.8830 - val_accuracy: 0.6483 - val_loss: 0.7990 - val_top_k_accuracy: 0.8683
Epoch 2/60
170/170 ━━━━━━━━━━━━━━━━━━━━ 73s 429ms/step - accuracy: 0.7074 - loss: 0.7082 - top_k_accuracy: 0.9071 - val_accuracy: 0.7042 - val_loss: 0.7210 - val_top_k_accuracy: 0.9124
Epoch 3/60
170/170 ━━━━━━━━━━━━━━━━━━━━ 72s 426ms/step - accuracy: 0.7109 - loss: 0.7242 - top_k_accuracy: 0.9149 - val_accuracy: 0.6431 - val_loss: 1.3089 - val_top_k_accuracy: 0.8801
Epoch 4/60
170/170 ━━━━━━━━━━━━━━━━━━━━ 73s 430ms/step - accuracy: 0.6979 - loss: 0.9460 - top_k_accuracy: 0.9120 - val_accuracy: 0.6071 - val_loss: 1.4983 - val_top_k_accuracy: 0.8344
Epoch 5/60
170/170 ━━━━━━━━━━━━━━━━━━━━ 72s 425ms/step - accuracy: 0.6992 - loss: 1.0046 - top_k_accuracy: 0.9027 - val_accuracy: 0.6512 - val_loss: 1.2222 - val_top_k_accuracy: 0.8904
Epoch 6/60
170/170 ━━━━━━━━━━━━━━━━━━━━ 74s 434ms/step - accuracy: 0.6946 -

## Evaluating Separate Models

In [15]:
resNet34Model.evaluate(testDataset)

72/72 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - accuracy: 0.7454 - loss: 1.1462 - top_k_accuracy: 0.9077


[1.169859528541565, 0.749780535697937, 0.9126426577568054]

In [16]:
efficientNetModel.evaluate(testDataset)

72/72 ━━━━━━━━━━━━━━━━━━━━ 24s 322ms/step - accuracy: 0.7105 - loss: 1600.7487 - top_k_accuracy: 0.9979


[1505.709228515625, 0.724319577217102, 0.998244047164917]

## Ensembling Both Models

In [19]:
input = tf.keras.layers.Input(shape=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"], 3))

y1 = resNet34Model(input)
y2 = efficientNetModel(input)

output = 0.5*y1 + 0.5*y2
ensembleModel = tf.keras.Model(inputs=input, outputs=output)

In [20]:
ensembleModel.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=CONFIGURATION["LEARNING_RATE"]),
    loss = lossFunction,
    metrics=METRICS
)

In [21]:
ensembleModel.evaluate(testDataset)

72/72 ━━━━━━━━━━━━━━━━━━━━ 37s 435ms/step - accuracy: 0.7237 - loss: 0.8546 - top_k_accuracy: 0.9729


[0.8374080061912537, 0.724319577217102, 0.9582967758178711]